# Toy Autonomous Agent Platform

This is a toy implementation of the Autonomous Agent Platform. It illustrates intent detection, planning (high level sketch, to-do list, and detailed plan or sub-plan), execution, and iterating.

## LLM Integration Points

This implementation shows the core workflow but uses hardcoded logic. In a real system, the following components would make calls to an LLM:

1. **Intent Detection** - LLM analyzes user query to determine intent
2. **High-Level Planning** - LLM generates initial task breakdown
3. **Detailed Planning** - LLM maps tasks to specific tools and agents
4. **Error Recovery** - LLM decides how to handle failures and replan
5. **Result Synthesis** - LLM formats final results for user consumption 

In [26]:
import random
from typing import List, Dict, Any


## LLM Integration Points - Intent Detection & Agent Selection

** LLM Calls in this section:**

1. **Intent Detection** - The `detect_intent()` function would make an LLM call:
   ```python
   response = llm.chat([{
       "role": "user", 
       "content": f"Analyze this user query and determine the intent: {query}"
   }])
   ```

2. **Agent Selection** - The `select_agents()` function could use LLM for dynamic selection:
   ```python
   response = llm.chat([{
       "role": "user", 
       "content": f"Given intent '{intent}' and available agents {[a.name for a in AGENTS]}, which agents should handle this task?"
   }])
   ```


In [27]:
# Mock Tools
def fetch_kpis(account: str) -> Dict[str, Any]:
    return {"account": account, "kpis": {"revenue": "$1M", "growth": "5%"}}

def fetch_support_tickets(account: str) -> Dict[str, Any]:
    return {"account": account, "tickets": ["Issue A - open", "Issue B - closed"]}

def draft_qbr_deck(account: str, kpis: Dict, tickets: List[str]) -> Dict[str, Any]:
    return {"deck": f"QBR Deck for {account}", "summary": f"{account} revenue {kpis['revenue']} with {len(tickets)} support issues."}


# Mock Agents
class Agent:
    def __init__(self, name, capabilities):
        self.name = name
        self.capabilities = capabilities  # list of tool names

    def __repr__(self):
        return f"Agent({self.name})"


retriever_agent = Agent("Retriever", ["fetch_kpis", "fetch_support_tickets"])
synthesizer_agent = Agent("Synthesizer", ["draft_qbr_deck"])

AGENTS = [retriever_agent, synthesizer_agent]

## LLM Integration Points - High-Level Planning

** LLM Call:** The `generate_high_level_sketch()` function would make an LLM call for intelligent task breakdown:

```python
response = llm.chat([{
    "role": "system",
    "content": "You are a task planning AI. Break down user requests into high-level steps."
}, {
    "role": "user", 
    "content": f"Create a high-level task breakdown for intent '{intent}'. Return a list of sequential steps needed to complete this task."
}])
```

The LLM would analyze the intent and generate contextually appropriate task sequences instead of using hardcoded mappings.


In [28]:
# Intent Detection
def detect_intent(query: str) -> str:
    if "QBR" in query:
        return "qbr_pack"
    elif "support" in query:
        return "support_summary"
    else:
        return "generic_task"


def select_agents(intent: str) -> List[Agent]:
    if intent == "qbr_pack":
        return [retriever_agent, synthesizer_agent]
    elif intent == "support_summary":
        return [retriever_agent]
    return AGENTS

## LLM Integration Points - Detailed Planning

** LLM Call:** The `generate_plan()` function would make an LLM call for tool selection and parameter mapping:

```python
response = llm.chat([{
    "role": "system",
    "content": f"You are a detailed planner. Map high-level tasks to specific tools. Available tools: {list(TOOL_REGISTRY.keys())}"
}, {
    "role": "user", 
    "content": f"Convert these high-level tasks {sketch} into executable steps using available tools for account '{account}'. Consider context: {context}"
}])
```

The LLM would:
- Map abstract tasks to concrete tools
- Determine proper argument passing
- Handle dependencies between steps
- Consider available context from previous executions


In [30]:
# -----------------------------
# Planner & DSL Plan Generator
# -----------------------------
def generate_high_level_sketch(intent: str) -> List[str]:
    print(f"PLANNING PHASE: Generating high-level sketch for intent '{intent}'")
    
    if intent == "qbr_pack":
        sketch = [
            "Fetch KPIs",
            "Fetch support tickets",
            "Draft QBR deck"
        ]
    elif intent == "support_summary":
        sketch = [
            "Fetch support tickets",
            "Summarize issues"
        ]
    else:
        sketch = ["Do generic retrieval", "Return results"]
    
    print(f"INITIAL PLAN CREATED: {len(sketch)} tasks identified")
    for i, task in enumerate(sketch, 1):
        print(f"   {i}. {task}")
    
    return sketch


## LLM Integration Points - Execution & Replanning

** LLM Calls during execution:**

1. **Error Recovery** - When tool execution fails:
```python
response = llm.chat([{
    "role": "user",
    "content": f"Tool '{tool_name}' failed with error: {error}. How should we recover? Available alternatives: {available_tools}"
}])
```

2. **Replanning Decision** - When deciding whether to replan:
```python
response = llm.chat([{
    "role": "user", 
    "content": f"Current status: {completed_tasks} done, {pending_tasks} pending. Context: {context}. Should we replan or continue?"
}])
```

3. **Result Synthesis** - Final response formatting:
```python
response = llm.chat([{
    "role": "user",
    "content": f"Synthesize these results into a user-friendly response: {results}"
}])
```


In [31]:
def generate_plan(sketch: List[str], agents: List[Agent], account: str, context: Dict[str, Any] = None) -> List[Dict]:
    """Generate a simple DSL-like plan based on sketch"""
    print(f"DETAILED PLANNING: Converting {len(sketch)} high-level tasks into executable plan")
    
    plan = []
    for i, step in enumerate(sketch, 1):
        print(f"   Planning step {i}: '{step}'")
        
        if "KPI" in step:
            tool_plan = {"tool": fetch_kpis, "args": {"account": account}}
            print(f"     -> Tool: {fetch_kpis.__name__}")
        elif "support" in step:
            tool_plan = {"tool": fetch_support_tickets, "args": {"account": account}}
            print(f"     -> Tool: {fetch_support_tickets.__name__}")
        elif "Draft" in step:
            tool_plan = {"tool": draft_qbr_deck, "args": {"account": account}}
            print(f"     -> Tool: {draft_qbr_deck.__name__}")
            print(f"     -> Dependencies: Will use context data from previous steps")
        else:
            print(f"     -> No specific tool mapped for this step")
            continue
            
        plan.append(tool_plan)
    
    print(f"PLAN GENERATED: {len(plan)} executable steps ready")
    return plan


In [32]:
def execute_plan(plan: List[Dict], context: Dict[str, Any], todo_list: List[str]) -> Dict[str, Any]:
    print(f"EXECUTION PHASE: Starting execution of {len(plan)} planned steps")
    print(f"TODO LIST STATUS: {len([t for t in todo_list if '[DONE]' not in t])} pending, {len([t for t in todo_list if '[DONE]' in t])} completed")
    
    results = {}
    for i, step in enumerate(plan):
        tool = step["tool"]
        args = step["args"].copy()
        
        print(f"\nEXECUTING STEP {i+1}/{len(plan)}")
        print(f"   Task: {todo_list[i]}")
        print(f"   Tool: {tool.__name__}")

        # Dynamically inject context values just before execution
        if tool == draft_qbr_deck:
            args["kpis"] = context.get("kpis", {"revenue": "N/A", "growth": "N/A"})
            args["tickets"] = context.get("tickets", [])
            print(f"   Dependencies injected: kpis={bool(context.get('kpis'))}, tickets={bool(context.get('tickets'))}")

        print(f"   Args: {args}")
        output = tool(**args)
        print(f"   SUCCESS: {tool.__name__} completed")

        # Store results in context for downstream use
        results.update(output)
        context.update(output)

        # Update the to-do list with detailed status
        old_task = todo_list[i]
        todo_list[i] = f"{old_task} [DONE]"
        print(f"TODO UPDATED: '{old_task}' -> '{todo_list[i]}'")
        
        # Show current todo list status
        pending_count = len([t for t in todo_list if '[DONE]' not in t])
        completed_count = len([t for t in todo_list if '[DONE]' in t])
        print(f"PROGRESS: {completed_count}/{len(todo_list)} tasks completed, {pending_count} remaining")
    
    print(f"\nEXECUTION COMPLETE: All {len(plan)} steps finished successfully")
    return results

In [33]:
def run(query: str, account: str):
    print(f"AUTONOMOUS AGENT STARTED")
    print(f"User Query: {query}")
    print(f"Account: {account}")
    print("=" * 80)

    # Step 1: Intent Detection
    print(f"\nSTEP 1: INTENT DETECTION")
    intent = detect_intent(query)
    print(f"Detected Intent: {intent}")

    # Step 2: Agent Selection
    print(f"\nSTEP 2: AGENT SELECTION")
    agents = select_agents(intent)
    print(f"Selected Agents: {agents}")

    # Step 3: High-Level Sketch / To-Do
    print(f"\nSTEP 3: INITIAL PLANNING")
    sketch = generate_high_level_sketch(intent)
    todo_list = sketch.copy()
    print(f"Initial To-Do List Created: {len(todo_list)} tasks")

    # Step 4: Iterative Planning + Execution
    print(f"\nSTEP 4: ITERATIVE EXECUTION LOOP")
    context = {}
    planning_iteration = 0
    
    while todo_list:
        planning_iteration += 1
        
        # Check current state and decide on planning approach
        pending_tasks = [task for task in todo_list if "[DONE]" not in task]
        completed_tasks = [task for task in todo_list if "[DONE]" in task]
        
        print(f"\nPLANNING ITERATION {planning_iteration}")
        print(f"Current Status: {len(completed_tasks)} completed, {len(pending_tasks)} pending")
        
        if planning_iteration == 1:
            print(f"INITIAL PLANNING: First-time planning for all {len(pending_tasks)} tasks")
        else:
            print(f"REPLANNING TRIGGERED")
            print(f"   Reason: Found {len(pending_tasks)} incomplete tasks after previous execution")
            print(f"   Previous context available: {list(context.keys())}")
            print(f"   Replanning strategy: Focus on remaining tasks with available context")
        
        # Generate plan for current pending tasks
        plan = generate_plan(pending_tasks, agents, account, context)
        
        # Execute the plan
        results = execute_plan(plan, context, todo_list)

        # Check for terminal conditions
        terminal_conditions = {
            "qbr_pack": "deck" in results,
            "support_summary": "tickets" in results,
            "generic_task": len(pending_tasks) == 0
        }
        
        if terminal_conditions.get(intent, False):
            print(f"\nTERMINAL CONDITION REACHED for intent '{intent}'")
            if intent == "qbr_pack":
                print("   QBR deck has been successfully created")
            elif intent == "support_summary":
                print("   Support tickets have been retrieved and summarized")
            else:
                print("   Generic task completed")
            break

        # Check if we need to replan
        still_pending = [task for task in todo_list if "[DONE]" not in task]
        if still_pending:
            print(f"\nREPLANNING DECISION:")
            print(f"   {len(still_pending)} tasks still pending: {still_pending}")
            print(f"   Will continue with replanning in next iteration")
            print(f"   Available context for next planning: {list(context.keys())}")
        else:
            print(f"\nALL TASKS COMPLETED - No replanning needed")
            break
        
        # Safety check to prevent infinite loops
        if planning_iteration > 5:
            print(f"\nSAFETY STOP: Maximum planning iterations ({planning_iteration}) reached")
            break

    # Step 5: Return response to client
    print(f"\nSTEP 5: FINAL RESULTS")
    print(f"Agent execution completed after {planning_iteration} planning iteration(s)")
    print(f"Final todo status: {len([t for t in todo_list if '[DONE]' in t])}/{len(todo_list)} completed")
    print("=" * 80)
    print("Final Response to Client:")
    for key, value in context.items():
        print(f"   {key}: {value}")
    print("=" * 80)


In [34]:
# Example Run
if __name__ == "__main__":
    run("Build a QBR pack for ACME", account="ACME Corp")

AUTONOMOUS AGENT STARTED
User Query: Build a QBR pack for ACME
Account: ACME Corp

STEP 1: INTENT DETECTION
Detected Intent: qbr_pack

STEP 2: AGENT SELECTION
Selected Agents: [Agent(Retriever), Agent(Synthesizer)]

STEP 3: INITIAL PLANNING
PLANNING PHASE: Generating high-level sketch for intent 'qbr_pack'
INITIAL PLAN CREATED: 3 tasks identified
   1. Fetch KPIs
   2. Fetch support tickets
   3. Draft QBR deck
Initial To-Do List Created: 3 tasks

STEP 4: ITERATIVE EXECUTION LOOP

PLANNING ITERATION 1
Current Status: 0 completed, 3 pending
INITIAL PLANNING: First-time planning for all 3 tasks
DETAILED PLANNING: Converting 3 high-level tasks into executable plan
   Planning step 1: 'Fetch KPIs'
     -> Tool: fetch_kpis
   Planning step 2: 'Fetch support tickets'
     -> Tool: fetch_support_tickets
   Planning step 3: 'Draft QBR deck'
     -> Tool: draft_qbr_deck
     -> Dependencies: Will use context data from previous steps
PLAN GENERATED: 3 executable steps ready
EXECUTION PHASE: Star